In [1]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# ───────────────────────────────────────────────────────────────
# 1) 파일 경로 설정 (본인 환경에 맞게 바꿔 주세요)
REG_DATA_FP   = 'data/module_datasets/regression.csv'
FEAT_FP       = 'data/df_feat.parquet'                     # ← 여기 변경
PRED_FP       = 'data/results/regression/regression_predictions_for_ensemble.csv'
CLF_MODEL_FP  = 'data/models/lgbm_classifier.pkl'
MASTER_CSV    = 'data/all_stocks_master.csv'

# ───────────────────────────────────────────────────────────────
# 2) 데이터 로드
df_reg  = pd.read_csv(REG_DATA_FP, parse_dates=["rcept_dt"], dtype={"stock_code":str})
df_feat = pd.read_parquet(FEAT_FP)                         # ← parquet 로드
df_pred = pd.read_csv(PRED_FP, parse_dates=["rcept_dt"], dtype={"stock_code":str})

# (필요하다면 타입 정리)
df_feat["stock_code"] = df_feat["stock_code"].astype(str).str.zfill(6)
df_feat["rcept_dt"]   = pd.to_datetime(df_feat["rcept_dt"])

# ───────────────────────────────────────────────────────────────
# 3) 분류 확률(p_up) 계산 
clf = joblib.load(CLF_MODEL_FP)
X_feat = df_feat.drop(columns=[
    "up_1d","corp_name","stock_code","rcept_dt","p_up"
], errors="ignore")
df_feat["p_up"] = clf.predict_proba(X_feat)[:, 1]

# ───────────────────────────────────────────────────────────────
# 4) 클러스터 라벨 생성
scaler = StandardScaler()
scaled_feats    = scaler.fit_transform(df_pred[["y_pred", "residual"]])
df_pred["cluster"] = KMeans(n_clusters=4, random_state=42, n_init=10).fit_predict(scaled_feats)

# ───────────────────────────────────────────────────────────────
# 5) 마스터 데이터 병합
#    - df_feat 에는 stock_code, rcept_dt, corp_name, p_up, ret_1d, up_1d 등
df_master = (
    df_pred
    .merge(df_feat[["stock_code","rcept_dt","corp_name","p_up","ret_1d"]],
           on=["stock_code","rcept_dt"], how="left")
    .merge(df_reg, on=["stock_code","rcept_dt"], how="left", suffixes=("","_orig"))
)

# ───────────────────────────────────────────────────────────────
# 6) CSV 저장
df_master.to_csv(MASTER_CSV, index=False, encoding="utf-8-sig")
print(f"✅ Master CSV saved → {MASTER_CSV}")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


✅ Master CSV saved → data/all_stocks_master.csv
